In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array
from time import time

In [2]:
# load data
df=pd.read_csv('PageBlocks_withoutdupl_02_v02.csv')  

del df['id']
del df['Unnamed: 0']

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,outlier
0,5.0,7.0,35.0,1.400,0.400,0.657,2.33,14.0,23.0,6.0,0
1,6.0,7.0,42.0,1.167,0.429,0.881,3.60,18.0,37.0,5.0,0
2,6.0,18.0,108.0,3.000,0.287,0.741,4.43,31.0,80.0,7.0,0
3,5.0,7.0,35.0,1.400,0.371,0.743,4.33,13.0,26.0,3.0,0
4,6.0,3.0,18.0,0.500,0.500,0.944,2.25,9.0,17.0,4.0,0


In [3]:
df.shape

(4982, 11)

In [4]:
# df to values
df = df.values

In [5]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/PageBlocks.json")  
gc = GCForest(config)

In [6]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,10]
X = df[:,0:10]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# count class
len(np.unique(y_train))

2

In [8]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-29 19:26:58,415][cascade_classifier.fit_transform] X_groups_train.shape=[(3487, 10)],y_train.shape=(3487,),X_groups_test.shape=[(1495, 10)],y_test.shape=(1495,)
[ 2018-04-29 19:26:58,416][cascade_classifier.fit_transform] group_dims=[10]
[ 2018-04-29 19:26:58,418][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 19:26:58,419][cascade_classifier.fit_transform] group_ends=[10]
[ 2018-04-29 19:26:58,420][cascade_classifier.fit_transform] X_train.shape=(3487, 10),X_test.shape=(1495, 10)
[ 2018-04-29 19:26:58,422][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3487, 10), X_cur_test.shape=(1495, 10)
[ 2018-04-29 19:26:59,293][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.86%
[ 2018-04-29 19:27:00,397][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.43%
[ 2018-04-29 19:27:01,493][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-29 19:27:09,094][cascade_classifier.transform] X_groups_test.shape=[(1495, 10)]
[ 2018-04-29 19:27:09,096][cascade_classifier.transform] group_dims=[10]
[ 2018-04-29 19:27:09,099][cascade_classifier.transform] X_test.shape=(1495, 10)
[ 2018-04-29 19:27:09,100][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1495, 10)


Test Accuracy of ExtratTrees = 99.264214 %


In [10]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-29 19:27:11,758][cascade_classifier.transform] X_groups_test.shape=[(1495, 10)]
[ 2018-04-29 19:27:11,760][cascade_classifier.transform] group_dims=[10]
[ 2018-04-29 19:27:11,762][cascade_classifier.transform] X_test.shape=(1495, 10)
[ 2018-04-29 19:27:11,763][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1495, 10)


In [11]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [12]:
X_test_new.shape

(1495, 12)

In [13]:
X_train_new.shape

(3487, 12)

In [14]:
# Cascade ExtraTrees
config = load_json("./examples/PageBlocksgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-29 19:27:14,313][cascade_classifier.fit_transform] X_groups_train.shape=[(3487, 12)],y_train.shape=(3487,),X_groups_test.shape=[(1495, 12)],y_test.shape=(1495,)
[ 2018-04-29 19:27:14,316][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-04-29 19:27:14,317][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 19:27:14,319][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-04-29 19:27:14,320][cascade_classifier.fit_transform] X_train.shape=(3487, 12),X_test.shape=(1495, 12)
[ 2018-04-29 19:27:14,322][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3487, 12), X_cur_test.shape=(1495, 12)
[ 2018-04-29 19:27:15,224][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.14%
[ 2018-04-29 19:27:16,333][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.14%
[ 2018-04-29 19:27:17,422][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-29 19:28:00,176][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=98.57%
[ 2018-04-29 19:28:01,270][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.57%
[ 2018-04-29 19:28:02,092][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=98.57%
[ 2018-04-29 19:28:03,024][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=97.99%
[ 2018-04-29 19:28:04,091][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.14%
[ 2018-04-29 19:28:05,171][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.56%
[ 2018-04-29 19:28:06,405][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.28%
[ 2018-04-29 19:28:07,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.43%


In [15]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [16]:
for i in range (20):
    t0=time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/PageBlocksgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt=time()-t0
    affiche()    
    print (" Time ",format(round(tt,3))," seconds") 

[ 2018-04-29 19:28:08,990][cascade_classifier.fit_transform] X_groups_train.shape=[(3487, 12)],y_train.shape=(3487,),X_groups_test.shape=[(1495, 12)],y_test.shape=(1495,)
[ 2018-04-29 19:28:08,991][cascade_classifier.fit_transform] group_dims=[12]
[ 2018-04-29 19:28:08,992][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-29 19:28:08,994][cascade_classifier.fit_transform] group_ends=[12]
[ 2018-04-29 19:28:08,995][cascade_classifier.fit_transform] X_train.shape=(3487, 12),X_test.shape=(1495, 12)
[ 2018-04-29 19:28:08,996][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(3487, 12), X_cur_test.shape=(1495, 12)
[ 2018-04-29 19:28:09,815][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.43%
[ 2018-04-29 19:28:10,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.43%
[ 2018-04-29 19:28:11,966][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

[ 2018-04-29 19:28:54,215][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=98.29%
[ 2018-04-29 19:28:55,325][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.28%
[ 2018-04-29 19:28:56,527][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.14%
[ 2018-04-29 19:28:57,611][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=97.99%
[ 2018-04-29 19:28:58,725][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.43%
[ 2018-04-29 19:28:59,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.28%
[ 2018-04-29 19:29:01,016][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.56%
[ 2018-04-29 19:29:02,130][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.14%


('Test Accuracy of Extratrees = 98.662207 %', 0, 'fois')
(' Time ', '65.726', ' seconds')


[ 2018-04-29 19:29:23,346][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.14%
[ 2018-04-29 19:29:24,494][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.14%
[ 2018-04-29 19:29:25,691][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.85%
[ 2018-04-29 19:29:26,744][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.28%
[ 2018-04-29 19:29:27,885][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.28%
[ 2018-04-29 19:29:29,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.70%
[ 2018-04-29 19:29:30,082][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.28%
[ 2018-04-29 19:29:31,163][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.14%


[ 2018-04-29 19:30:14,146][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.28%
[ 2018-04-29 19:30:15,234][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.85%
[ 2018-04-29 19:30:16,254][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.43%
[ 2018-04-29 19:30:17,441][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.14%
[ 2018-04-29 19:30:17,687][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.62%
[ 2018-04-29 19:30:17,688][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=98.60%
[ 2018-04-29 19:30:17,690][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.62%
[ 2018-04-29 19:30:17,692][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=98.60%
[ 2018-04-29 19:30:17,694][

...

[ 2018-04-29 19:55:39,039][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.56%
[ 2018-04-29 19:55:40,143][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.28%
[ 2018-04-29 19:55:41,077][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.14%
[ 2018-04-29 19:55:42,101][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.14%
[ 2018-04-29 19:55:42,345][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.80%
[ 2018-04-29 19:55:42,346][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=99.13%
[ 2018-04-29 19:55:42,348][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.80%
[ 2018-04-29 19:55:42,350][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=99.13%
[ 2018-04-29 19:55:42,351][

('Test Accuracy of Extratrees = 99.130435 %', 18, 'fois')
(' Time ', '76.305', ' seconds')


[ 2018-04-29 19:56:15,222][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.71%
[ 2018-04-29 19:56:16,324][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.43%
[ 2018-04-29 19:56:17,424][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.85%
[ 2018-04-29 19:56:18,465][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.99%
[ 2018-04-29 19:56:19,601][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.57%
[ 2018-04-29 19:56:20,602][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.99%
[ 2018-04-29 19:56:21,677][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.85%
[ 2018-04-29 19:56:22,759][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.85%


[ 2018-04-29 19:57:04,889][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.56%
[ 2018-04-29 19:57:06,021][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.28%
[ 2018-04-29 19:57:06,949][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.85%
[ 2018-04-29 19:57:07,933][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.14%
[ 2018-04-29 19:57:08,194][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=98.71%
[ 2018-04-29 19:57:08,196][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=99.00%
[ 2018-04-29 19:57:08,198][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=98.71%
[ 2018-04-29 19:57:08,199][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=99.00%
[ 2018-04-29 19:57:08,200][

('Test Accuracy of Extratrees = 98.996656 %', 19, 'fois')
(' Time ', '53.815', ' seconds')


# ('Test Accuracy of Extratrees = 99.264214 %', 14, 'fois')

# (' Time ', '1149.7', ' seconds')